In [2]:
import pandas as pd
df=pd.read_csv(r"E:\train.csv")

In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [15]:

df = df.dropna().reset_index(drop=True)


X = df.drop('label', axis=1)
y = df['label']

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
import numpy as np

voc_size = 5000

messages = X.copy()

import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []

messages.reset_index(drop=True, inplace=True)

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


onehot_repr = [one_hot(words, voc_size) for words in corpus]

sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)

X_final = np.array(embedded_docs)

y_final = np.array(y)
print(f"Length of X_final: {len(X_final)}, Length of y_final: {len(y_final)}")

# If the lengths are still inconsistent, you can manually drop mismatched rows
if len(X_final) != len(y_final):
    min_len = min(len(X_final), len(y_final))
    X_final = X_final[:min_len]
    y_final = y_final[:min_len]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


Length of X_final: 18285, Length of y_final: 18285


In [16]:

# Create model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
print(model.summary())

# Finally train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)


C:\Users\Jenith\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.7410 - loss: 0.4422 - val_accuracy: 0.9181 - val_loss: 0.2053
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9511 - loss: 0.1354 - val_accuracy: 0.9198 - val_loss: 0.2079
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9753 - loss: 0.0799 - val_accuracy: 0.9162 - val_loss: 0.2275
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9798 - loss: 0.0619 - val_accuracy: 0.9082 - val_loss: 0.2734
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9858 - loss: 0.0510 - val_accuracy: 0.9059 - val_loss: 0.2592
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9912 - loss: 0.0333 - val_accuracy: 0.9024 - val_loss: 0.3388
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9917 - loss: 0.0295 - val_accuracy: 0.9056 - val_loss: 0.3726
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9942 - loss: 0.0203 - val

In [17]:
import numpy as np
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


array([[3105,  314],
       [ 293, 2323]], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.8994200497100249

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))